In [1]:
import pandas as pd, numpy as np, os, glob
import re
pd.options.mode.chained_assignment = None

In [2]:
Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')
Test = pd.read_csv('/Users/monoaki/vs_code/亞東/encrypt/baseline_01_sample_500-test_periHD.csv')
IntraHD_Test = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/IntraHD_ExternalData_05-17.csv')
order_data = pd.read_csv('../../../encrypt/backup/Order2016_20_en.csv')

/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33553/1801650475.py:1: DtypeWarning: Columns (6,7,8,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_2693_PeriHD.csv')
/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33553/1801650475.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  IntraHD_Train = pd.read_csv('../../../Dialysis_Hypotension/IDH_full_freetext_data/final_filter_train_full_1124_IntraHD.csv')
/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33553/1801650475.py:5: DtypeWarning: Columns (16,18,20,26) have mixed types. Specify dtype option on import or set low_memory=False.
  order_data = pd.read_csv('../../../encrypt/backup/Order2016_20_en.csv')


In [3]:
full_DataFrame = pd.read_csv('../../../encrypt/periHD2016_20_en.csv')

/var/folders/bh/sd4z8w4n5_z9223mbfs7x9fr0000gn/T/ipykernel_33553/3472545216.py:1: DtypeWarning: Columns (3,5,6,7,8,9,10,11,15,16,17,18,25,26,27,28,29,30,31,41,45,46,47,48,49,50,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  full_DataFrame = pd.read_csv('../../../encrypt/periHD2016_20_en.csv')


In [4]:
IntraHD_Train['處置其他+症狀處置'] = IntraHD_Train['處置其他']+'||'+IntraHD_Train['症狀處置']
print(IntraHD_Train.shape)
IntraHD_Train = IntraHD_Train.drop_duplicates(subset=['處置其他+症狀處置'])
print(IntraHD_Train.shape)
print(Train.shape)
Train = Train.drop_duplicates(subset=['處置其他結束'])
# Train = pd.concat([Train,Train.sample(307, random_state=123)])
Train['透析時間長度'] = pd.to_datetime(Train['透析時間結束'])-pd.to_datetime(Train['透析開始時間'])
Test['透析時間長度'] = pd.to_datetime(Test['透析時間結束'])-pd.to_datetime(Test['透析開始時間'])
print(Train.shape)
print(Test.shape)

(1452843, 25)
(1124, 25)
(61160, 64)
(2386, 65)
(500, 66)


In [5]:
Train_ID = Train['ID'].astype(str).drop_duplicates()
Test_ID = Test['ID'].astype(str).drop_duplicates()
overall_ID = pd.concat([Train_ID,Test_ID]).drop_duplicates()
print(Train_ID.shape, Test_ID.shape, overall_ID.shape)
full_DataFrame_Train = pd.merge(Train_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Test = pd.merge(Test_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Overall = pd.merge(overall_ID, full_DataFrame, on='ID', how='inner')
full_DataFrame_Overall['ID'] = full_DataFrame_Overall['ID'].astype(str)
# full_DataFrame_Overall['透析時間長度'] = (pd.to_datetime(full_DataFrame_Overall['透析時間結束']).dt.to_period('H') - pd.to_datetime(full_DataFrame_Overall['透析開始時間']).dt.to_period('H'))
full_DataFrame_Overall['洗腎紀錄時間2'] = pd.to_datetime(full_DataFrame_Overall['洗腎紀錄時間']).dt.to_period('D')

(1259,) (357,) (1382,)


In [6]:
# Follow-up duration, month, median (IQR)

full_DataFrame_Overall = full_DataFrame_Overall.sort_values(by=['ID','洗腎紀錄時間2'])
IQR_dict_overall = {'ID':[],'IQR':[]}
for id_ in full_DataFrame_Overall['ID'].drop_duplicates():
    filter_ID_df = full_DataFrame_Overall[full_DataFrame_Overall['ID']==id_].reset_index()
    filter_ID_df['洗腎紀錄時間2'] = filter_ID_df['洗腎紀錄時間2']
    IQR_len = len(filter_ID_df)
    IQR_dict_overall['ID'].append(str(id_))
    IQR_dict_overall['IQR'].append(int(re.findall(r'\d+', str(filter_ID_df['洗腎紀錄時間2'][IQR_len-1] - filter_ID_df['洗腎紀錄時間2'][0]))[0])/30)
IQR_dict_overall = pd.DataFrame(IQR_dict_overall)
IQR_dict_internal = pd.merge(IQR_dict_overall, Train_ID, on='ID', how='inner')
IQR_dict_external = pd.merge(IQR_dict_overall, Test_ID, on='ID', how='inner')

In [8]:
First_dcit = pd.DataFrame([], columns=full_DataFrame_Overall.columns.to_list())
for idx, ID_ in enumerate(overall_ID.to_list()):
    First_Filter_ID_ = full_DataFrame_Overall[full_DataFrame_Overall['ID']==ID_]
    First_Filter_ID_['洗腎紀錄時間'] = pd.to_datetime(First_Filter_ID_['洗腎紀錄時間']).copy(deep=True)
    First_Filter_DateTime = First_Filter_ID_.sort_values(by=['洗腎紀錄時間'])
    Get_First_Records = First_Filter_DateTime.iloc[0].to_frame().transpose()
    First_dcit = pd.concat([First_dcit, Get_First_Records])

In [9]:
# 基本資料 Variables
age_df = pd.read_csv('../../../encrypt/Base2016_20_en.csv')
age_df_overall = pd.merge(age_df, First_dcit, on='ID', how='inner')
age_df_overall['生日'] = pd.to_datetime(age_df_overall['生日']).copy(deep=True)
age_df_overall['age'] = ((age_df_overall['洗腎紀錄時間']-age_df_overall['生日'])/365).astype(str).str.split(' days').str[0]
# 洗腎時間
order_data_overall = pd.merge(order_data[['ID','小時/次']],overall_ID,on='ID',how='inner')
order_data_overall['小時/次'] = order_data_overall['小時/次'].apply(pd.to_numeric, errors='coerce').dropna()
order_data_overall = order_data_overall.dropna()
print('order_data_overall: ',order_data_overall.shape)
# Laboratory results
lab_df = pd.concat([pd.read_csv('../../../encrypt/backup/Lab2016_18_en.csv'),pd.read_csv('../../../encrypt/backup/Lab2019_20_en.csv')])
lab_df['報告日'] = pd.to_datetime(lab_df['報告日']).dt.to_period('D')
lab_df2 = pd.concat([pd.read_csv('../../../encrypt/backup/Lab_1_en.csv'),pd.read_csv('../../../encrypt/backup/Lab_2_en.csv')])

order_data_overall:  (40484, 2)


In [10]:
First_dict_AV = First_dcit[['ID','FISTULA','GRAFT','DLC']]

In [11]:
lab_df_get_first_datetime = lab_df.sort_values(by=['ID','報告日'])
lab_df_get_first_datetime['ID'] = lab_df_get_first_datetime['ID'].astype(str)
print(lab_df_get_first_datetime.shape)
lab_df_get_first_datetime = pd.merge(First_dcit['ID'], lab_df_get_first_datetime, on='ID', how='inner')
print(lab_df_get_first_datetime.shape)
lab_df_get_first_datetime2 = lab_df2.sort_values(by=['ID','報告日'])
lab_df_get_first_datetime2['ID'] = lab_df_get_first_datetime2['ID'].astype(str)
print(lab_df_get_first_datetime2.shape)
lab_df_get_first_datetime2 = pd.merge(First_dcit['ID'], lab_df_get_first_datetime2, on='ID', how='inner')
print(lab_df_get_first_datetime2.shape)

(1272247, 4)
(925848, 4)
(1648126, 5)
(1092971, 5)


In [12]:
# merge 篩選區
age_df_overall = age_df_overall
age_df_Internal = pd.merge(Train_ID, age_df_overall, on='ID', how='inner')
age_df_External =pd.merge(Test_ID, age_df_overall, on='ID', how='inner')
print(age_df_overall.shape, age_df_Internal.shape, age_df_External.shape)
lab_df_overall = lab_df_get_first_datetime
lab_df_Internal = pd.merge(Train_ID, lab_df_overall, on='ID', how='inner')
lab_df_External =pd.merge(Test_ID, lab_df_overall, on='ID', how='inner')
print(lab_df_overall.shape, lab_df_Internal.shape, lab_df_External.shape)

(1382, 68) (1259, 68) (357, 68)
(925848, 4) (795253, 4) (469708, 4)


In [13]:

def check_lab2_value(filter_df, name=None):
    if 'HCV' in [name]:
        check_ = filter_df[filter_df['檢驗名稱'].str.contains('HCV')]['DATANUM2'].values
        if len(check_)==0:
            return 0
        check_ = re.sub(r'\d', '', check_[0])
        # print(check_)
        if '-' not in check_ or 'on' not in check_:
            return 1
        else:
            return 0
    
    elif len(filter_df)==1:
        return float(filter_df[0])
    else:
        return 0 
lab_df_get_first_datetime2 = lab_df_get_first_datetime2.sort_values(by=['ID','報告日'])
lab2_first_dict = {'ID':[],'HBs Ag':[],'Anti-HBc':[],'Anti-HBs':[],'Anti-HCV':[],'BUN':[],'BUN (洗腎後)':[],'HCV_cls':[]}
for idx, id in enumerate(lab_df_get_first_datetime2.ID.drop_duplicates()):
    filter_2 = None
    filter_1 = lab_df_get_first_datetime2[lab_df_get_first_datetime2['ID']==id]
    filter_2 = filter_1[(filter_1['檢驗名稱']=='HBs Ag')|(filter_1['檢驗名稱']=='Anti-HBc')|
                        (filter_1['檢驗名稱']=='Anti-HBs')|(filter_1['檢驗名稱']=='Anti-HCV')|
                        (filter_1['檢驗名稱']=='BUN')|(filter_1['檢驗名稱']=='BUN (洗腎後)')]
    filter_2['DATANUM2'] = filter_2['DATANUM'].replace(r'\d', '')
    filter_2['DATANUM'] = filter_2['DATANUM'].replace(r'[^0-9.]', '', regex=True)
    filter_2 = filter_2.drop_duplicates(subset=['檢驗名稱'])
    if filter_2.shape[0]!=0:
        lab2_first_dict['ID'].append(id)
        lab2_first_dict['HBs Ag'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='HBs Ag']['DATANUM'].values))
        lab2_first_dict['Anti-HBc'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='Anti-HBc']['DATANUM'].values))
        lab2_first_dict['Anti-HBs'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='Anti-HBs']['DATANUM'].values))
        lab2_first_dict['Anti-HCV'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='Anti-HCV']['DATANUM'].values))
        lab2_first_dict['BUN'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='BUN']['DATANUM'].values))
        lab2_first_dict['BUN (洗腎後)'].append(check_lab2_value(filter_2[filter_2['檢驗名稱']=='BUN (洗腎後)']['DATANUM'].values))
        lab2_first_dict['HCV_cls'].append(check_lab2_value(filter_2, 'HCV'))
    # if idx==50:
    #     break
# lab2_first_dict

In [14]:
lab2_first_df = pd.DataFrame(lab2_first_dict)
lab2_first_df = lab2_first_df.rename(columns={'HBs Ag':'HBsAg','Anti-HBc':'HBc','Anti-HBs':'HBs','Anti-HCV':'HCV'})

In [15]:
for i in range(len(lab2_first_df['ID'])):
    if lab2_first_df['HBsAg'][i]==0:
        lab2_first_df['HBsAg'][i] = 0
    else:
        lab2_first_df['HBsAg'][i] = 1
    if lab2_first_df['HBc'][i]==0:
        lab2_first_df['HBc'][i] = 0
    else:
        lab2_first_df['HBc'][i] = 1
    
    if lab2_first_df['HBs'][i]==0:
        lab2_first_df['HBs'][i] = 0
    else:
        lab2_first_df['HBs'][i] = 1

In [16]:
lab2_first_df['HBV']=np.nan
for i in range(len(lab2_first_df['HBV'])):
    if (lab2_first_df['HBV'][i] == 1) |(lab2_first_df['HBV'][i] == 0):
        continue
    if lab2_first_df['HBs'][i] == 1:
        lab2_first_df['HBV'][i] = 0
    elif lab2_first_df['HBs'][i] == 0:
        if lab2_first_df['HBc'][i] == 1:
            lab2_first_df['HBV'][i] = 1
        elif lab2_first_df['HBc'][i] == 0:
            lab2_first_df['HBV'][i] = 0
        elif lab2_first_df['HBsAg'][i] == 1:
            lab2_first_df['HBV'][i] = 1
        else:
            lab2_first_df['HBV'][i] = 0
    if lab2_first_df['HBsAg'][i] == 1:
        lab2_first_df['HBV'][i] = 1
    elif lab2_first_df['HBsAg'][i] == 0:
        lab2_first_df['HBV'][i] = 0

In [17]:
lab2_first_df['URR%'] = round((lab2_first_df['BUN']-lab2_first_df['BUN (洗腎後)'])/lab2_first_df['BUN'] * 100,1)
lab2_neg_df = lab2_first_df[lab2_first_df['URR%']<0]
lab2_neg_df['URR%'] = round((lab2_neg_df['BUN (洗腎後)']-lab2_neg_df['BUN'])/lab2_neg_df['BUN (洗腎後)'] * 100,1)
lab2_first_df.update(lab2_neg_df)
lab2_first_df_overall = lab2_first_df
lab2_first_df_internal = pd.merge(Train_ID, lab2_first_df_overall, on='ID', how='inner')
lab2_first_df_external = pd.merge(Test_ID, lab2_first_df_overall, on='ID', how='inner')
print(lab2_first_df_overall.shape, lab2_first_df_internal.shape, lab2_first_df_external.shape)

(1302, 10) (1181, 10) (354, 10)


In [24]:
# lab_dict = {'WBC count, 10^3/μL':[],'Hemoglobin, g/ dL':[],'Hematocrit, %':[],'MCV, fL':[],'MCH, pg':[],'MCHC, g/dL':[],
#             'PLT count, 10^3/ μL':[],'Calcium, mg/dL':[],'Phosphate, mg/ dL':[],'Glucose, mg/dL':[],'BUN, mg/dL':[],'Creatinine, mg/ dL':[],
#             'Albumin, g/dL':[], 'Electrolyte Sodium, mmol/L':[],'Potassium, mmol/L':[], 'Serum iron, μg/ dL':[], 'Ferritin, ng/mL':[],
#             'Parathyroid hormone, pg/ mL':[],'Urea Reduction Rate, %':[],'ALT (SGPT)':[],'IRON/TIBC':[],'HBV, n (%)':[],'HCV, n (%)':[],}
lab_dict = {'WBC count, 10^3/μL':[],'Hemoglobin, g/ dL':[],'Hematocrit, %':[],'MCV, fL':[],'MCH, pg':[],'MCHC, g/dL':[],
            'PLT count, 10^3/ μL':[],'Calcium, mg/dL':[],'Phosphate, mg/ dL':[],'Glucose, mg/dL':[],'BUN, mg/dL':[],'Creatinine, mg/ dL':[],
            'Albumin, g/dL':[], 'Electrolyte Sodium, mmol/L':[],'Potassium, mmol/L':[],'Serum iron, μg/ dL':[], 'Ferritin, ng/mL':[],
            'Parathyroid hormone, pg/ mL':[],'Urea Reduction Rate, %':[],'ALT (SGPT)':[],'IRON/TIBC':[],'HBV, n (%)':[],'HCV, n (%)':[]}
dataframe = lab_df_External
dataframe = dataframe.sort_values(by=['ID','報告日'])
def get_lab_value(dataframe, name='WBC'):
    if name in ['Glucose']:
        value_list = dataframe[dataframe['檢驗名稱'].str.contains(name)].drop_duplicates(subset='ID')
    else:
        value_list = dataframe[dataframe['檢驗名稱']==name].drop_duplicates(subset='ID')
    value_list = value_list.dropna()
    value_list = value_list.apply(pd.to_numeric, errors='coerce')
    return "{} ({})".format(round(np.mean(value_list['DATANUM']),1),round(np.std(value_list['DATANUM']),1))
lab_list = ['WBC','HGB','HCT','MCV','MCH','MCHC','Platelet','Ca','P  (Phosphorus)','Glucose','BUN','Creatinine','ALBUMIN','Na','K',
            'Iron','Ferritin','Intact PTH','URR%','ALT (SGPT)','IRON/TIBC','HBV','HCV']
print(len(lab_dict))
print(len(lab_list))
for idx,lab_type in enumerate(lab_list):
    lab_dict_col = list(lab_dict.keys())
    if lab_type in ['URR%','HBV','HCV']:
        pass
    else:
        lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_Internal, name = lab_type))
        lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_External, name = lab_type))
        lab_dict[lab_dict_col[idx]].append(get_lab_value(lab_df_overall, name = lab_type))

23
23


In [25]:
lab2_first_df_internal

,ID,HBsAg,HBc,HBs,HCV,BUN,BUN (洗腎後),HCV_cls,HBV,URR%
0,7844676,0.0,0.0,0.0,0.000,79.0,0.0,0.0,0.0,100.0
1,97766575,1.0,1.0,1.0,0.030,75.0,28.0,0.0,1.0,62.7
2,04E34S555,1.0,1.0,1.0,0.036,72.0,0.0,0.0,1.0,100.0
3,02312W131,1.0,1.0,1.0,0.046,97.0,0.0,0.0,1.0,100.0
4,05E86R555,1.0,1.0,1.0,0.035,49.0,12.0,0.0,1.0,75.5
...,...,...,...,...,...,...,...,...,...,...
1176,63415232,1.0,1.0,1.0,0.039,45.0,0.0,0.0,1.0,100.0
1177,04641U666,1.0,0.0,0.0,0.039,0.0,0.0,0.0,1.0,NaN
1178,060F95505,1.0,1.0,1.0,0.041,62.0,12.0,0.0,1.0,80.6
1179,072868H28,1.0,1.0,1.0,0.043,139.0,0.0,0.0,1.0,100.0


In [26]:
lab_dict['Urea Reduction Rate, %']=["{} ({})".format(round(np.mean(lab2_first_df_internal['URR%']),1),round(np.std(lab2_first_df_internal['URR%']),1)),
                                     "{} ({})".format(round(np.mean(lab2_first_df_external['URR%']),1),round(np.std(lab2_first_df_external['URR%']),1)),
                                     "{} ({})".format(round(np.mean(lab2_first_df_overall['URR%']),1),round(np.std(lab2_first_df_overall['URR%']),1))]
lab_dict['HBV, n (%)']=["{} ({}%)".format(len(lab2_first_df_internal[lab2_first_df_internal['HBV']==1]),round(len(lab2_first_df_internal[lab2_first_df_internal['HBV']==1])/len(lab2_first_df_internal)*100,1)),
                        "{} ({}%)".format(len(lab2_first_df_external[lab2_first_df_external['HBV']==1]),round(len(lab2_first_df_external[lab2_first_df_external['HBV']==1])/len(lab2_first_df_external)*100,1)),
                        "{} ({}%)".format(len(lab2_first_df_overall[lab2_first_df_overall['HBV']==1]),round(len(lab2_first_df_overall[lab2_first_df_overall['HBV']==1])/len(lab2_first_df_overall)*100,1)),]
lab_dict['HCV, n (%)']=["{} ({}%)".format(len(lab2_first_df_internal[lab2_first_df_internal['HCV_cls']==1]),round(len(lab2_first_df_internal[lab2_first_df_internal['HCV_cls']==1])/len(lab2_first_df_internal)*100,1)),
                        "{} ({}%)".format(len(lab2_first_df_external[lab2_first_df_external['HCV_cls']==1]),round(len(lab2_first_df_external[lab2_first_df_external['HCV_cls']==1])/len(lab2_first_df_external)*100,1)),
                        "{} ({}%)".format(len(lab2_first_df_overall[lab2_first_df_overall['HCV_cls']==1]),round(len(lab2_first_df_overall[lab2_first_df_overall['HCV_cls']==1])/len(lab2_first_df_overall)*100,1)),]

In [27]:
# ,"Potassium","Urea_Reduction_Rate",'HBV','HCV']

In [28]:
def per_hours(overall, internal_id, external_id):
    inter = pd.merge(internal_id, overall, on='ID', how='inner')
    exter = pd.merge(external_id, overall, on='ID', how='inner')
    over = overall['小時/次']
    inter = inter['小時/次']
    exter = exter['小時/次']
    return [f"{round(np.mean(over),1)} ({round(np.std(over),1)})",
            f"{round(np.mean(inter),1)} ({round(np.std(inter),1)})",
            f"{round(np.mean(exter),1)} ({round(np.std(exter),1)})"]


def weight_calc(dataframe):
    dataframe_pos = dataframe[['體重1開始','體重機器顯示','體重結束']]
    dataframe_pos = dataframe_pos.apply(pd.to_numeric, errors='coerce')
    dataframe_pos = dataframe_pos.dropna()
    dataframe_pos = dataframe_pos.reset_index()
    dataframe_pos['洗後體重'] = dataframe_pos['體重1開始']-dataframe_pos['體重機器顯示']
    for i in range(len(dataframe_pos)):
        if dataframe_pos.loc[i,'洗後體重'] <30:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
        if dataframe_pos.loc[i,'洗後體重'] >100:
            dataframe_pos.loc[i,'洗後體重'] = dataframe_pos.loc[i,'體重結束']
    dataframe_pos['乾體重'] = dataframe_pos['體重結束']
    return dataframe_pos

def VA(dataframe):
    FISTULA = "{} ({}%)".format(len(dataframe['FISTULA'].dropna()),round(len(dataframe['FISTULA'].dropna())/len(dataframe)*100,1))
    GRAFT = "{} ({}%)".format(len(dataframe['GRAFT'].dropna()),round(len(dataframe['GRAFT'].dropna())/len(dataframe)*100,1))
    Catheter = dataframe[(dataframe['DLC'].notna())|(dataframe['PERMCATH'].notna())][['DLC','PERMCATH']]
    Catheter = "{} ({}%)".format(len(Catheter),round(len(Catheter)/len(dataframe)*100,1))
    return [FISTULA, GRAFT, Catheter]

def get_BP(dataframe):
    dataframe = dataframe[['NEW開始血壓SBP','NEW開始血壓DBP','開始脈搏','結束血壓SBP','結束血壓DBP','結束脈搏']]
    dataframe = dataframe.apply(pd.to_numeric, errors='coerce')
    dataframe = dataframe.dropna().astype(float)
    dataframe = dataframe[(dataframe['NEW開始血壓SBP']>=40) & (dataframe['NEW開始血壓SBP']<=240) & (dataframe['結束血壓SBP']>=40) & (dataframe['結束血壓SBP']<=240)]
    dataframe = dataframe[(dataframe['NEW開始血壓DBP']>=30) & (dataframe['NEW開始血壓DBP']<=140) & (dataframe['結束血壓DBP']>=30) & (dataframe['結束血壓DBP']<=140)]
    return dataframe

Table_1_Variables = {"Variables":['Participants, n', 'Follow-up duration, month, median (IQR)', 'Age, years', 'Men, n(%)', 'Weight, kg'],
                     "Internal Data":[1259, 
                                      f"{round(IQR_dict_internal['IQR'].describe()['mean'],1)} ({round(IQR_dict_internal['IQR'].describe()['25%'],1)}-{round(IQR_dict_internal['IQR'].describe()['75%'],1)})",
                                      f"{round(np.mean(age_df_Internal['age'].astype(int).values),1)} ({round(age_df_Internal['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_Internal['性別']=='男')} ({round(sum(age_df_Internal['性別']=='男')/len(age_df_Internal)*100,1)}%)",
                                      f"{round(np.mean(age_df_Internal['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_Internal['體重1開始'].astype(float).std(),1)})"
                                      ],
                     "External Data":[357,
                                      f"{round(IQR_dict_external['IQR'].describe()['mean'],1)} ({round(IQR_dict_external['IQR'].describe()['25%'],1)}-{round(IQR_dict_external['IQR'].describe()['75%'],1)})",
                                      f"{round(np.mean(age_df_External['age'].astype(int).values),1)} ({round(age_df_External['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_External['性別']=='男')} ({round(sum(age_df_External['性別']=='男')/len(age_df_External)*100,1)}%)",
                                      f"{round(np.mean(age_df_External['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_External['體重1開始'].astype(float).std(),1)})"
                                      ],
                     "Overall Data":[1382,
                                     f"{round(IQR_dict_overall['IQR'].describe()['mean'],1)} ({round(IQR_dict_overall['IQR'].describe()['25%'],1)}-{round(IQR_dict_overall['IQR'].describe()['75%'],1)})",
                                      f"{round(np.mean(age_df_overall['age'].astype(int).values),1)} ({round(age_df_overall['age'].astype(int).std(),1)})",
                                      f"{sum(age_df_overall['性別']=='男')} ({round(sum(age_df_overall['性別']=='男')/len(age_df_overall)*100,1)}%)",
                                      f"{round(np.mean(Test['體重1開始'].fillna(0).astype(float)),1)} ({round(Test['體重1開始'].astype(float).std(),1)})"
                                      ]}

# 暫時移除Blood flow rate：來源於IntraHD
Table_1_Dialysis_information = {"Variables":['Hours per session, hour','Dry body wieght, kg','Pre-dialysis weight, kg','post-dialysis, kg'],
                                "Internal Data":[per_hours(order_data_overall, Train_ID, Test_ID)[1],
                                                 f"{round(np.mean(weight_calc(age_df_Internal)['乾體重']),1)} ({round(np.std(weight_calc(age_df_Internal)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_Internal['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_Internal['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_Internal)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_Internal)['洗後體重']),1)})",
                                                 ],
                                "External Data":[per_hours(order_data_overall, Train_ID, Test_ID)[2],
                                                 f"{round(np.mean(weight_calc(age_df_External)['乾體重']),1)} ({round(np.std(weight_calc(age_df_External)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_External['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_External['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_External)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_External)['洗後體重']),1)})",
                                                 ],
                                "Overall Data":[per_hours(order_data_overall, Train_ID, Test_ID)[0],
                                                 f"{round(np.mean(weight_calc(age_df_overall)['乾體重']),1)} ({round(np.std(weight_calc(age_df_overall)['乾體重']),1)})",
                                                 f"{round(np.mean(age_df_overall['體重1開始'].fillna(0).astype(float)),1)} ({round(age_df_overall['體重1開始'].astype(float).std(),1)})",
                                                 f"{round(np.mean(weight_calc(age_df_overall)['洗後體重']),1)} ({round(np.std(weight_calc(age_df_overall)['洗後體重']),1)})",
                                                 ]} 

Table_1_Vascular_access = {"Variables": ['AV-Fistula, n(%)','AV-Graft, n(%)','Catheter, n(%)'],
                           "Internal Data":VA(age_df_overall),
                           "External Data":VA(age_df_Internal),
                           "Overall Data":VA(age_df_External)}

Table_1_Blood_pressures = {"Variables":['Start-SBP, mmHg','Start-DBP, mmHg','Start-HR, beat/min','End-SBP, mmHg','End-DBP, mmHg','End-HR, beat/min'],
                           "Internal Data":[f"{round(np.mean(get_BP(age_df_Internal)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_Internal)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_Internal)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_Internal)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_Internal)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_Internal)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_Internal)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_Internal)['結束脈搏']),1)})",
                                            ],
                           "External Data":[f"{round(np.mean(get_BP(age_df_External)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_External)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_External)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_External)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_External)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_External)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_External)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_External)['結束脈搏']),1)})",
                                            ],
                           "Overall Data":[f"{round(np.mean(get_BP(age_df_overall)['NEW開始血壓SBP']),1)} ({round(np.std(get_BP(age_df_overall)['NEW開始血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['NEW開始血壓DBP']),1)} ({round(np.std(get_BP(age_df_overall)['NEW開始血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['開始脈搏']),1)} ({round(np.std(get_BP(age_df_overall)['開始脈搏']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束血壓SBP']),1)} ({round(np.std(get_BP(age_df_overall)['結束血壓SBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束血壓DBP']),1)} ({round(np.std(get_BP(age_df_overall)['結束血壓DBP']),1)})",
                                            f"{round(np.mean(get_BP(age_df_overall)['結束脈搏']),1)} ({round(np.std(get_BP(age_df_overall)['結束脈搏']),1)})",
                                            ]}

In [29]:
pd.concat([pd.DataFrame(Table_1_Variables),pd.DataFrame(Table_1_Vascular_access),pd.DataFrame(Table_1_Dialysis_information), pd.DataFrame(Table_1_Blood_pressures),pd.DataFrame(lab_dict).transpose().reset_index().rename(columns={'index':'Variables',0:'Internal Data',1:'External Data', 2:'Overall Data'})], axis=0)

,Variables,Internal Data,External Data,Overall Data
0,"Participants, n",1259,357,1382
1,"Follow-up duration, month, median (IQR)",21.6 (1.9-38.1),40.4 (27.5-60.3),22.8 (2.2-40.0)
2,"Age, years",64.6 (13.1),61.0 (11.8),64.3 (13.0)
3,"Men, n(%)",705 (56.0%),208 (58.3%),786 (56.9%)
4,"Weight, kg",59.6 (19.1),60.5 (18.2),63.0 (14.2)
0,"AV-Fistula, n(%)",663 (48.0%),595 (47.3%),213 (59.7%)
1,"AV-Graft, n(%)",133 (9.6%),124 (9.8%),26 (7.3%)
2,"Catheter, n(%)",523 (37.8%),483 (38.4%),123 (34.5%)
0,"Hours per session, hour",3.7 (0.5),3.7 (0.6),3.7 (0.6)
1,"Dry body wieght, kg",60.3 (16.6),62.7 (14.6),61.4 (16.2)
